In [1]:
import warnings
warnings.filterwarnings("ignore")

import os

import pandas as pd

In [2]:
buyers_transaction_df = pd.read_csv("../data/cleaned/buyers_transaction.csv")

display(buyers_transaction_df.head())

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
display(buyers_transaction_df[["quantity", "unitprice"]].describe())

,quantity,unitprice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


**Observation:** We can see that the `quantity` and `unitprice` values are negative. This raises a question that **_Why these values are negative?_**

## Inspecting negative `quantity` and `unitprice` values

### 1. When `quantity < 0`

In [4]:
negative_qty_df = buyers_transaction_df[buyers_transaction_df["quantity"] < 0]
negative_qty_df["first_char_invoiceno"] = negative_qty_df["invoiceno"].str[0]

display(negative_qty_df.head())

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,first_char_invoiceno
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,C
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,C
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,C
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,C
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,C


#### Different values of first character in `invoiceno`

In [5]:
negative_qty_df["first_char_invoiceno"].unique()

array(['C', '5'], dtype=object)

#### a. Count of `customerid` when first char is 'C'

In [6]:
first_char_C = negative_qty_df[negative_qty_df["first_char_invoiceno"] == "C"]

first_char_C.customerid.nunique()

1589

#### b. Count of `customerid` when first char is '5'

In [7]:
first_char_5 = negative_qty_df[negative_qty_df["first_char_invoiceno"] == "5"]

first_char_5.customerid.nunique()

0

#### c. `unitprice` when first char is 'C'

In [8]:
sorted(first_char_C["unitprice"].unique(), reverse=True)[:5]

[38970.0, 17836.46, 16888.02, 16453.71, 13541.33]

#### d. `unitprice` when first char is '5'

In [9]:
first_char_5["unitprice"].unique()

array([0.])

### 2. When `unitprice < 0`

In [10]:
negative_unit_price_df = buyers_transaction_df[buyers_transaction_df["unitprice"] < 0]

display(negative_unit_price_df.head())

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


### Conclusion

#### 1. When `quantity < 0`

* For negative `quantity` values we can see that the `invoiceno` has `C` as a prefix. This usually be the case either the buyer returned or cancelled the order for a certain amount of quantities.

* When first character of `invoiceno` is **'C'** we can see that there are 1589 different `customerid` with varying `unitprice`

* When first character of `invoiceno` is **'5'** the `customerid` column has missing values and unitprice is also `0`

---

#### 2. When `unitprice < 0`

* For negative `unitprice` the `invoiceno` has `A` as a prefix. We can observe that description has value `Adjust bad debt`. 

## Inspecting missing values

In [36]:
buyers_transaction_copy = buyers_transaction_df.copy(deep=True)

buyers_transaction_copy["description"] = buyers_transaction_copy.description.astype(str)

In [37]:
buyers_transaction_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   invoiceno    541909 non-null  object 
 1   stockcode    541909 non-null  object 
 2   description  541909 non-null  object 
 3   quantity     541909 non-null  int64  
 4   invoicedate  541909 non-null  object 
 5   unitprice    541909 non-null  float64
 6   customerid   406829 non-null  float64
 7   country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [12]:
buyers_transaction_df[buyers_transaction_df.description.isna()]

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [17]:
buyers_transaction_df.loc[buyers_transaction_df["stockcode"] == "22139", "description"].value_counts()

RETROSPOT TEA SET CERAMIC 11 PC     988
amazon                                1
Name: description, dtype: int64

In [23]:
'buyers_transaction_df.loc[buyers_transaction_df["stockcode"] == "22145"]

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
19025,537827,22145,CHRISTMAS CRAFT HEART STOCKING,30,2010-12-08 14:43:00,2.1,15332.0,Lithuania
20392,538041,22145,NaN,30,2010-12-09 13:09:00,0.0,NaN,United Kingdom


In [25]:
buyers_transaction_df[buyers_transaction_df["customerid"] == 15332.0]

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
7986,537081,22409,MONEY BOX BISCUITS DESIGN,12,2010-12-05 12:00:00,1.25,15332.0,Lithuania
7987,537081,21915,RED HARMONICA IN BOX,36,2010-12-05 12:00:00,1.25,15332.0,Lithuania
7988,537081,15056BL,EDWARDIAN PARASOL BLACK,12,2010-12-05 12:00:00,5.95,15332.0,Lithuania
7989,537081,15056N,EDWARDIAN PARASOL NATURAL,12,2010-12-05 12:00:00,5.95,15332.0,Lithuania
7990,537081,20679,EDWARDIAN PARASOL RED,12,2010-12-05 12:00:00,5.95,15332.0,Lithuania
7991,537081,20686,DOLLY MIXTURE CHILDREN'S UMBRELLA,6,2010-12-05 12:00:00,3.25,15332.0,Lithuania
7992,537081,20682,RED RETROSPOT CHILDRENS UMBRELLA,6,2010-12-05 12:00:00,3.25,15332.0,Lithuania
7993,537081,20681,PINK POLKADOT CHILDRENS UMBRELLA,6,2010-12-05 12:00:00,3.25,15332.0,Lithuania
7994,537081,21429,RED GINGHAM ROSE JEWELLERY BOX,8,2010-12-05 12:00:00,1.65,15332.0,Lithuania
7995,537081,21914,BLUE HARMONICA IN BOX,36,2010-12-05 12:00:00,1.25,15332.0,Lithuania


In [44]:
buyers_transaction_df.groupby(["description"]).agg({"unitprice": "nunique", "unitprice": "unique"})

,unitprice
description,
4 PURPLE FLOCK DINNER CANDLES,"[2.55, 5.06, 4.96, 0.79]"
50'S CHRISTMAS GIFT BAG LARGE,"[1.25, 2.46, 1.04]"
DOLLY GIRL BEAKER,"[1.25, 2.46, 1.08]"
I LOVE LONDON MINI BACKPACK,"[4.15, 3.75, 8.29]"
I LOVE LONDON MINI RUCKSACK,[4.15]
...,...
wrongly marked carton 22804,[0.0]
wrongly marked. 23343 in box,[0.0]
wrongly sold (22719) barcode,[0.0]


In [48]:
buyers_transaction_copy[
                      buyers_transaction_copy["description"] == "WHITE HANGING HEART T-LIGHT HOLDER"
                     ]

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom
...,...,...,...,...,...,...,...,...
537291,581246,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1,2011-12-08 10:59:00,2.95,15453.0,United Kingdom
537326,581253,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2,2011-12-08 11:15:00,2.95,16891.0,United Kingdom
537852,581356,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2011-12-08 12:28:00,2.95,17830.0,United Kingdom
539979,581452,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2011-12-08 18:03:00,2.55,17675.0,United Kingdom


In [ ]:
buyers_transaction_copy

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [22]:
buyers_transaction_df.loc[buyers_transaction_df["stockcode"] == "22145", "stockcode"].value_counts()

22145    3
Name: stockcode, dtype: int64